In [2]:
# 02_feature_engineering.ipynb
# Author: Onur Balic
# Description: Advanced feature engineering & node-level energy analysis

# ==========================
# Import necessary libraries
# ==========================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Set visualization styles
sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (12, 6)

# ==========================
# Load dataset
# ==========================
df = pd.read_csv("../data/butter_e_energy.csv")

# Convert timestamp to datetime
df["timestamp"] = pd.to_datetime(df["timestamp"])

# Show basic info
print("Shape:", df.shape)
print("\nColumns:", df.columns.tolist())
print("\nDate range:", df["timestamp"].min(), "to", df["timestamp"].max())
df.head()

Shape: (1059206, 3)

Columns: ['timestamp', 'node', 'watts']

Date range: 2022-09-22 16:28:01+00:00 to 2022-09-30 09:30:01+00:00


,timestamp,node,watts
0,2022-09-24 08:20:01+00:00,r103u05,721
1,2022-09-24 08:21:01+00:00,r103u05,470
2,2022-09-24 08:22:01+00:00,r103u05,461
3,2022-09-24 08:23:01+00:00,r103u05,458
4,2022-09-24 08:24:01+00:00,r103u05,462
